Implementing Hill Climbing with Random Restarts
Objective: Implement a hill climbing algorithm using random restarts.

Problem Description: You are tasked with implementing a simple optimization algorithm called hill climbing. The goal is to find the maximum value of a given function within a specified range. If the algorithm reaches a local maximum, it should randomly restart from a new starting point.

Key components:

- start_point : Initial (x,y) coordinates
- R : Number of random restarts (100)
- I : Number of iterations per restart (1000)
- perturbation_step : How much to perturb the solution in each step (0.5)

The algorithm works as follows:

1. Start from an initial point
2. For each of the R restarts:
   - For each of the I iterations:
     - Generate a candidate solution by randomly perturbing the current solution
     - If the candidate solution is better, update the best solution
     - If we're stuck in a local maximum (no improvement for half the iterations), restart from a random point
   - Keep track of the best solution found across all restarts
Hill climbing is a local search algorithm that always moves in the direction of increasing value. The random restarts help escape local optima by occasionally jumping to a completely new location in the search space.

In [ ]:
import math
import random

# Define the Rosenbrock function
def f(x, y):
    a = 1
    b = 100
    return (a - x)**2 + b * (x**2 - y)**2

# Perform hill climbing with random restarts
def hill_climbing_with_restarts(start_point, R=100, I=1000, perturbation_step=0.5):
    best_solution = None
    
    for _ in range(R):
        current_x, current_y = start_point
        best_x, best_y = start_point
        
        for _ in range(I):
            # Generate a candidate solution by perturbing the current solution
            candidate_x = current_x + (random.uniform(-1, 1) * perturbation_step)
            candidate_y = current_y + (random.uniform(-1, 1) * perturbation_step)
            
            if f(candidate_x, candidate_y) > f(best_x, best_y):
                best_x, best_y = candidate_x, candidate_y
            
            # Check for local maximum
            # If we’re in the second half of the iterations (_ > I // 2) and the current function value is extremely close to the best-known value (within 1e-5 difference)
            if _ > I // 2 and abs(f(current_x, current_y) - f(best_x, best_y)) < 1e-5:
                start_point = (random.uniform(-3, 3), random.uniform(-3, 3))
                break
        
        if best_solution is None or f(best_x, best_y) > f(best_solution[0], best_solution[1]):
            best_solution = (best_x, best_y)
    
    return best_solution

# Example usage
start_point = (0, 0)
result = hill_climbing_with_restarts(start_point)
print(f"Best solution found: {result}")


Best solution found: (-0.4991755614037574, -0.4974330806217897)


Regularized Objective Function for Knapsack Problem
Objective: Modify the objective function of the knapsack problem to include a penalty term that accounts for constraint violation.

Problem Description: Given a set of items with weights and values, you want to maximize the total value without exceeding a given weight limit. The new objective function should penalize solutions that exceed the weight limit.
```python
def F(v, c=1.0):
    return f(v) - c * h(v)
```
Key components:

- f(v) : Original objective function (maximize total value)
- h(v) : Constraint penalty function (penalizes solutions that exceed weight limit)
- F(v, c) : Regularized objective function that combines value and penalty
- c is a regularization parameter that controls the strength of the penalty
This approach is called "penalty method" or "regularization" because:

1. It transforms a constrained optimization problem into an unconstrained one
2. It allows the algorithm to explore solutions that might temporarily violate constraints
3. As the parameter c increases, solutions that violate constraints are penalized more heavily
The penalty function h(v) typically measures how much the total weight exceeds the maximum allowed weight. For example:
h(v) = max(0, total_weight(v) - max_weight)
This regularization approach allows the search algorithm to move through infeasible regions of the search space, which can sometimes lead to better final solutions than if we strictly enforced the constraint at every step.

The algorithm:

1. Define items with values and weights
2. Define a maximum weight constraint
3. Use hill climbing with random restarts to find the best solution
4. For each iteration, flip one item's inclusion status (0 to 1 or 1 to 0)
5. Accept the change if it improves the regularized objective function
The regularization approach allows the algorithm to explore solutions that might slightly violate the weight constraint during the search, but eventually converge to a valid solution due to the penalty term.

In [2]:
import random

# Define the knapsack problem parameters
items = [{'value': 60, 'weight': 10}, {'value': 100, 'weight': 20}, {'value': 120, 'weight': 30}]
max_weight = 50

# Define the original objective function
def f(v):
    total_value = 0
    for i in range(len(items)):
        if v[i]:
            total_value += items[i]['value']
    return total_value

# Define the constraint penalty function
def h(v):
    total_weight = sum(items[i]['weight'] * v[i] for i in range(len(items)))
    return max(total_weight - max_weight, 0)

# Define the regularized objective function
def F(v, c=1.0):
    return f(v) - c * h(v)

# Perform hill climbing with random restarts
def hill_climbing_with_restarts(R=100, I=1000):
    best_solution = None
    
    for _ in range(R):
        current_solution = [random.choice([0, 1]) for _ in range(len(items))]
        best_solution = current_solution
        
        for _ in range(I):
            candidate_solution = current_solution[:]
            i = random.randint(0, len(items) - 1)
            candidate_solution[i] = 1 - candidate_solution[i]
            
            if F(candidate_solution) > F(best_solution):
                best_solution = candidate_solution
    
    return best_solution

# Example usage
result = hill_climbing_with_restarts()
print(f"Best solution found: {result}")


Best solution found: [0, 0, 1]


Simulated Annealing for Traveling Salesman Problem (TSP)
Objective: Implement simulated annealing to solve the traveling salesman problem.

Problem Description: Given a set of cities and distances between them, find the shortest possible route that visits each city exactly once and returns to the starting point. Use simulated annealing to minimize the total distance.

Key components:

- T0 : Initial temperature (1000)
- alpha : Cooling rate (0.95)
- I : Number of iterations per temperature (1000)
The algorithm:

1. Generate a random initial tour
2. For each temperature level:
   - For each iteration:
     - Generate a candidate tour by swapping two random cities
     - Calculate the change in distance (delta_D)
     - If delta_D < 0 (improvement), accept the change
     - If delta_D >= 0 (worse), accept with probability e^(-delta_D/T)
   - Reduce temperature by multiplying by alpha
Simulated annealing is inspired by the annealing process in metallurgy. The "temperature" parameter controls how likely the algorithm is to accept worse solutions. At high temperatures, it frequently accepts worse solutions, allowing it to explore the search space broadly. As the temperature decreases, it becomes more selective, eventually converging to a good solution.

The key advantage of simulated annealing over hill climbing is its ability to escape local optima by occasionally accepting worse solutions, especially early in the search process.


## Temperature in Simulated Annealing for TSP
The term "temperature" in simulated annealing comes from the physical annealing process in metallurgy that inspired the algorithm:

1. Physical Annealing : In metallurgy, annealing involves heating a material to a high temperature and then slowly cooling it to reduce defects and increase strength.
2. Algorithm Analogy : In the algorithm, "temperature" is a control parameter that determines how likely the algorithm is to accept worse solutions:
   
   - At high temperatures, the algorithm frequently accepts worse solutions, allowing it to explore the search space broadly and escape local optima.
   - As the temperature decreases, the algorithm becomes more selective, eventually only accepting improvements.
The probability of accepting a worse solution is calculated using the formula:
P(accept) = e^(-delta_D/T)
Where:

- delta_D is the increase in distance (how much worse the new solution is)
- T is the current temperature
This means:

- When T is high, even large increases in distance have a good chance of being accepted
- When T is low, only small increases have any chance of being accepted
- As T approaches 0, the algorithm essentially becomes a hill climbing algorithm
The temperature schedule (starting temperature T0, cooling rate alpha) is crucial for the algorithm's performance. If cooled too quickly, the algorithm may get stuck in local optima; if cooled too slowly, it may take too long to converge.

In [ ]:
import random

# Example distance matrix for 4 cities
D = [
    [0, 1, 2, 3],
    [1, 0, 4, 5],
    [2, 4, 0, 6],
    [3, 5, 6, 0]
]

# Calculate the total distance of a tour
def total_distance(tour):
    n = len(tour)
    dist = D[tour[-1]][tour[0]]
    for i in range(n - 1):
        dist += D[tour[i]][tour[i + 1]]
    return dist

# Generate a random initial tour
def generate_initial_tour(n):
    cities = list(range(n))
    random.shuffle(cities)
    return cities

# Simulated Annealing Algorithm
def simulated_annealing(T0, alpha, I):
    n = len(D)
    current_tour = generate_initial_tour(n)
    best_tour = current_tour
    T = T0
    
    for _ in range(I):
        for _ in range(I):
            candidate_tour = current_tour[:]
            i, j = random.sample(range(n), 2)
            candidate_tour[i], candidate_tour[j] = candidate_tour[j], candidate_tour[i]
            
            delta_D = total_distance(candidate_tour) - total_distance(current_tour)
            if delta_D < 0 or random.random() < 1 / T:
                current_tour = candidate_tour
                if total_distance(candidate_tour) < total_distance(best_tour):
                    best_tour = candidate_tour
        
        T *= alpha
    
    return best_tour

# Example usage
T0 = 1000
alpha = 0.95
I = 1000
result = simulated_annealing(T0, alpha, I)
print(f"Best solution found: {result}")


Best solution found: [0, 1, 2, 3]
